## Object Interceptions with Soft Actor-Critic Model

### Drake simulation setup code

In [1]:
# Import libraries and functions for drake simulation
import numpy as np
import os
import random

import pydot
from pydrake.all import (LeafSystem,Sphere, Box )

from manipulation import running_as_notebook
import matplotlib.pyplot as plt
from IPython.display import HTML, SVG, display

from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.math import RollPitchYaw


from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation.scenarios import MakeManipulationStation


from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem
from pydrake.systems.framework import LeafSystem

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad
from pydrake.all import (ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         PiecewisePolynomial, PiecewiseQuaternionSlerp,
                         RigidTransform, RollPitchYaw, RotationMatrix,
                         Simulator, Solve, StartMeshcat, TrajectorySource)
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial
from pydrake.trajectories import PiecewisePolynomial

# Create a Drake temporary directory to store files.
# Note: this tutorial will create two temporary files (cylinder.sdf and
# table_top.sdf) under `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

# Press the 'Stop JointSliders' button in MeshCat to continue.
iiwa0 = FindResourceOrThrow(
    "drake/manipulation/models/"
    "iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
# model_inspector(iiwa7_model_file)

### PROJECT SDF FILES ###

#Project XML file under Models Folder  
project_xml_file = os.path.join("/work/Models", "project.xml")
project_xml = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
"""

with open(project_xml_file, "w") as f:
    f.write(project_xml)


### GOALIE END-EFFECTOR
goalie_end_sdf_file = os.path.join("/work/Models", "goalie_end.sdf")
goalie_end_sdf = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="goalie_end">
    <link name="goalie_end_link">
    
      <visual name="visual">
        <pose>0 0 0.15 0 0 0</pose>
        <geometry>
          <box>
            <size>.30 .06 0.30</size>
          </box>
        </geometry>
       
      </visual>
      
      <collision name="collision">
        <pose>0 0 .15 0 0 0</pose>
        <geometry>
          <box>
            <size>.30 .06 0.30</size>
          </box>
        </geometry>
      </collision>
    </link>
    <frame name="goalie_end_center">
      <pose relative_to="goalie_end_link">0 0 0.15 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(goalie_end_sdf_file, "w") as f:
    f.write(goalie_end_sdf)

### GOAL POST:
goalpost_sdf_file = os.path.join("/work/Models", "goalpost.sdf")
goalpost_sdf = """
<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="goalpost">

    <link name="goalpost_link">

      <visual name="visual">
        <pose>0 0 0 0 0 3.1415693</pose>
        <geometry>
            <mesh>
                <uri>/work/soccerpostfixed.obj</uri>
                <scale> 0.02 0.02 0.02 </scale>
            </mesh>
        </geometry>
      </visual>

      <collision name="collision">
         <pose>0.05  -0.4 0.6 0 0 0</pose>
        <geometry>
            <box>
            <size>2 0.1 1.2</size>
          </box>
        </geometry>
      </collision>

    </link>

    <frame name="goalpost_center">
      <pose relative_to ="goalpost_link">0 0 0.50 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(goalpost_sdf_file, "w") as f:
    f.write(goalpost_sdf)



### Floor:
floor_sdf_file = os.path.join("/work/Models", "floor.sdf")
floor_sdf = """
<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="floor">

    <link name="floor_link">

      <visual name="visual">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
            <box>
                <size>10 20 0.1</size>
            </box>
        </geometry>
      </visual>

      <collision name="collision">
         <pose>0 0 0 0 0 0</pose>
        <geometry>
            <box>
            <size>10 20 0.1</size>
          </box>
        </geometry>
      </collision>

    </link>

    <frame name="floor_center">
      <pose relative_to ="floor_link">0 0 .05 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(floor_sdf_file, "w") as f:
    f.write(floor_sdf)


### SOCCER BALL
Soccer_Ball_sdf_file = os.path.join("/work/Models", "Ball.sdf")
Soccer_Ball_sdf = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf xmlns:xacro="http://www.ros.org/wiki/xacro" version="1.7">
  <model name="Soccer Ball">
    <link name="Soccer_Ball_link">

      <inertial>
        <pose>0 0 0.15 0 0 0</pose>
        <mass>0.5</mass>
        <inertia>
          <ixx>0.008</ixx> 
          <iyy>0.008</iyy>
          <izz>0.008</izz>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyz>0</iyz>
        </inertia>
      </inertial>


      <visual name="visual">
        <pose>0 0 0.15 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>/work/Ball.obj</uri>
            <scale> .1 .1 .1 </scale>
          </mesh>
        </geometry>

        <material>
         <diffuse>0.9 0.8 0.7 1.0</diffuse>
        </material>
      </visual>


      <collision name="collision">
        <pose>0 0 .15 0 0 0</pose>
        <geometry>
          <sphere>
            <radius>.1</radius>
          </sphere>
        </geometry>


        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            10000
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            0.5
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>


      </collision>
    </link>
    <frame name="Soccer_ball_center">
      <pose relative_to="Soccer_Ball_link">0 0 0 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(Soccer_Ball_sdf_file, "w") as f:
    f.write(Soccer_Ball_sdf)
    

# MODEL DIRECTIVES
model_directives = """
    directives:
    
    

    - add_model:
        name: iiwa
        file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
        default_joint_positions:
            iiwa_joint_1: [0]
            iiwa_joint_2: [0]
            iiwa_joint_3: [1.5]
            iiwa_joint_4: [1.5]
            iiwa_joint_5: [0]
            iiwa_joint_6: [1.57]
            iiwa_joint_7: [0]
         
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, 0]}
    - add_model:
        name: floor
        file: package://project/floor.sdf
    - add_weld:
        parent: world
        child: floor_center
        X_PC:
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, 0]}
    


    - add_model:
        name: goalie_end
        file: package://project/goalie_end.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: goalie_end::goalie_end_link
        X_PC:
            translation: [0, 0, .04]
            rotation: !Rpy { deg: [0, 0, 90]}
    - add_model:
        name: goalpost
        file: package://project/goalpost.sdf
    - add_weld:
        parent: floor_center
        child: goalpost_center
        X_PC:
            translation: [0, -0.8, 0.5]
            rotation: !Rpy { deg: [0, 0, 0]}
    - add_model:
        name: Soccer_Ball
        file: package://project/Ball.sdf
        default_free_body_pose:
            Soccer_Ball_link:
                translation: [0, 5, 0]


    """

INFO:drake:Meshcat listening for connections at https://c79dd6be-c791-4b96-b0a1-e37f965290d0.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


### Randomized Ball Trajectory Code

In [2]:
initial_soccer_params = {
    "s_ball_goalpost": 3,
    "desired_approach_speed": np.random.uniform(7,12), 
    "x_range": 0.80,
    "z_range": 0.90
}

#ALSO RETURNS TIME OF FLIGHT FOR PROJECTILE:
def get_desired_velocity(desired_pos, params = initial_soccer_params):
    """
    Given final x and z position of the ball when it enters the goal post, 
    returns Velocity vector with which ball should be launched from the initial position 
    """
    s,vy_des  = params["s_ball_goalpost"],params["desired_approach_speed"]
    x_des,_,z_des = desired_pos

    tau = s/vy_des #Time for ball to reach the goal post 
    vx_des = x_des/tau #No external force in x-direction
    vz_des = (z_des + 1/2*9.81*tau**2)/tau
    return (vx_des, -vy_des, vz_des), tau # vy = -vy because of the way we are simulating the env.  
    

def randomize_ball_trajectory_w_time(params = initial_soccer_params):
    x_rand = random.uniform(-params["x_range"], params["x_range"])
    z_rand = random.uniform(0, params["z_range"])
    desired_pos = (x_rand,0.0, z_rand)
   
    v, tau = get_desired_velocity(desired_pos)

    return v, desired_pos,tau

randomize_ball_trajectory_w_time()

((2.6244299345015505, -11.74972963356471, 3.500903526986059),
 (0.6700826358602774, 0.0, 0.5741070700371125),
 0.2553250239418352)

### Initial Setup

In [3]:
import gym
import numpy as np
import math
import torch
from pydrake.all import StartMeshcat
from gym import spaces
from collections import deque

from stable_baselines3.common.env_checker import check_env

from manipulation.utils import LoadDataResource, running_as_notebook
from manipulation.envs.box_flipup import BoxFlipUpEnv

from psutil import cpu_count
num_cpu = int(cpu_count() / 2) if running_as_notebook else 2

# Optional imports (these are heavy dependencies for just this one notebook)
sb3_available = False
try:
    from stable_baselines3 import SAC
    from stable_baselines3.common.vec_env import SubprocVecEnv
    from stable_baselines3.common.env_util import make_vec_env
    sb3_available = True
except ImportError:
    print("stable_baselines3 not found")
    print("Consider 'pip3 install stable_baselines3'.")


### GYM ENVIRONMENT:

In [4]:
class SoccerEnv(gym.Env):
    """Custom Environment that follows gym interface"""

    def __init__(self):
        super(SoccerEnv, self).__init__()

        #Action space is delta of Joint Position: 7 DOF 
        self.action_space = spaces.Box(
            low= -math.pi/90, 
            high= math.pi/90,
            shape=(7,), 
            dtype=np.float32
            )

        
        #Soccer X,Y,Z
        #End-effector X,Z 
        #Last Joint Pose 

        self.observation_space = spaces.Box(
                        low= np.append(np.array([-20, -20, -1.0,-0.3]),  np.full(7, -math.pi)), 
                        high=  np.append(np.array([20,20, 1.0,1.0]), np.full(7, math.pi)), shape=(11,), dtype=np.float32
                        )


        
        self.params = initial_soccer_params

        self.sim_time_step = 0.01

        self.builder, self.station, self.plant, self.diagram, self.simulator, self.context, self.plant_context,self.q0, self.station_context= self.helper_initialize_soccer_env()
        self.total_ep_reward = 0
         

    def step(self, action):
        '''
        Agent takes action -> New Observation and reward 
        '''
        # print("delPosition: ",action)
        self.JP += action

        #Call helper function to actually step through drake simulation:
        end_effector_XZ, ball_XZ, self.intercepted, rpy = self.helper_step_DrakeSimulation(JointPosition = self.JP)
        
        


        #Observation:
        ob = np.append(ball_XZ,end_effector_XZ)
        observation = np.append(ob, self.JP) 
     

        #Reward:
        x_e,y_e = end_effector_XZ
        x_b, y_b = ball_XZ   
        r_0, y_0 = self.default_rpy[0], self.default_rpy[2]
        r_cur, y_cur = rpy[0],rpy[2]
        reward_orientation = -np.sqrt((r_0 - r_cur)**2 + (y_0 - y_cur)**2) 


        self.reward =  - np.sqrt((x_e - x_b)**2 + (y_e - y_b)**2)*10 + int(self.intercepted)*1000 + reward_orientation #Reward for smaller Distance 
        self.total_ep_reward += self.reward
        # print("Orientation:", reward_orientation/ self.reward)
        #Info:
        info = {}


        #Done:
        self.current_time += self.sim_time_step
        self.done = self.current_time > self.terminal_time + 0.2

        if self.intercepted:
            print("Intercepted?", self.intercepted )
        # print(self.current_time)
        # print("Ball: ", ball_XZ)
        # print("End-effector:", end_effector_XZ)
        
        # print("\n")
        return observation, self.reward, self.done, info


    def reset(self):
        '''
        Reset method
        '''
        #Initialize parameters and pose:

        print("Total Ep. Reward: ", self.total_ep_reward)
        self.done = False
        self.total_ep_reward = 0 

        self.JP = self.q0.copy() 

        end_effector_XZ, ball_XZ,tau = self.helper_reset_episode()
        self.terminal_time = tau
        self.current_time = 0

        #Base Observation:
        base_observation =   np.append(np.append(ball_XZ,end_effector_XZ), self.q0.copy()) 
        # print(tau)
        return base_observation  # reward, done, info can't be 

    def render(self, mode = "human"):
        self.visualizer.StartRecording(False)
        # self.simulator.AdvanceTo(self.simulator.get_context().get_time()+self.sim_time_step)
        self.visualizer.PublishRecording()

    def close (self):
        pass




    def helper_initialize_soccer_env(self, params = initial_soccer_params):
        """
        Helper function that initializes Soccerbot Environment withh class instance is created 
        
        """
        meshcat.Delete()
        meshcat.DeleteAddedControls()

        builder = DiagramBuilder()
        station = builder.AddSystem(
            MakeManipulationStation(
                model_directives, package_xmls=["/work/Models/project.xml"], time_step =self.sim_time_step
                )
            )

        plant = station.GetSubsystemByName("plant")
        controller_plant = station.GetSubsystemByName(
            "iiwa_controller").get_multibody_plant_for_control()
        scene_graph = station.GetSubsystemByName('scene_graph')
        #station.SetupManipulationClassStation()

        # Add a meshcat visualizer.
        visualizer = MeshcatVisualizer.AddToBuilder(
           builder, station.GetOutputPort("query_object"), meshcat)
        self.visualizer = visualizer
        AddMultibodyTriad(
            plant.GetFrameByName("goalie_end_link"), scene_graph)
        #station.Finalize()


        diagram = builder.Build()
        simulator = Simulator(diagram)
        context = simulator.get_mutable_context()
        station_context = station.GetMyMutableContextFromRoot(context)
        plant_context = plant.GetMyMutableContextFromRoot(context)
        q0 = np.array([0,0,1.57,1.57,0,1.57,0])


        simulator.AdvanceTo(.001)

        end_effector = plant.GetBodyByName("goalie_end_link")
        end_pose_RP = plant.EvalBodyPoseInWorld(plant_context, end_effector)
        end_effector_pose = end_pose_RP.translation()
        end_effector_rotation = end_pose_RP.rotation()
        rpy = RollPitchYaw(end_effector_rotation).vector()

        self.default_rpy = rpy
        return builder, station, plant, diagram, simulator, context, plant_context,q0, station_context



    def helper_reset_episode(self):
        """
            - Set Ball and Manipulator to default position 
            - send ball at random trajectory Toward goal post 

            - Returns initial end-effector pose and ball pose 
        """
        v, desired_pos, tau = randomize_ball_trajectory_w_time()
        vx, vy, vz = v
        xf, yf, zf = desired_pos
        soccer_ball_model = self.plant.GetModelInstanceByName('Soccer_Ball')
        self.plant.SetPositions(self.plant_context,soccer_ball_model, np.array([1,0,0,0,0,self.params["s_ball_goalpost"]+0.35, 0]))
        self.plant.SetVelocities(self.plant_context,soccer_ball_model, np.array([0,0,0,vx,vy,vz]))

        iiwa_model = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName('iiwa'), self.q0)
    


        end_effector = self.plant.GetBodyByName("goalie_end_link")
        
        end_effector_pose = self.plant.EvalBodyPoseInWorld(self.plant_context, end_effector).translation()
        # print(end_effector_pose)
    

        e_x, e_y, e_z = tuple(end_effector_pose)
        end_effector_XZ  = np.array([end_effector_pose[0], end_effector_pose[2]])
        
        ball_XZ = np.zeros(2)
        
        return end_effector_XZ, ball_XZ, tau




    def helper_step_DrakeSimulation(self,JointPosition):
        '''
        Helper function for step method:
            - Given joint angles, steps single step in simulation
            - Return current end-effector pose and ball pose 
        '''
        JointLim = [2.97,1.2, 2.97, 1.85, 2.75, 2.05, 3.09]

        for i in range(7):
            if JointPosition[i] < -JointLim[i]: JointPosition[i] = -JointLim[i]
            if JointPosition[i] >  JointLim[i]: JointPosition[i] = JointLim[i]
    
        iiwa_model = self.plant.GetModelInstanceByName("iiwa")
        self.station.GetInputPort('iiwa_position').FixValue(self.station_context, JointPosition)


        ########

        end_effector = self.plant.GetBodyByName("goalie_end_link")
        end_effector_pose = self.plant.EvalBodyPoseInWorld(self.plant_context, end_effector).translation()
        end_effector_rotation = self.plant.EvalBodyPoseInWorld(self.plant_context, end_effector).rotation()
        rpy = RollPitchYaw(end_effector_rotation).vector()


        e_x, e_y, e_z = tuple(end_effector_pose)
        end_effector_XZ  = np.array([end_effector_pose[0], end_effector_pose[2]])


        soccer_ball_model = self.plant.GetModelInstanceByName('Soccer_Ball')
        soccer_current_pose = self.plant.GetPositions(self.plant_context, soccer_ball_model) # 7 elements: 4 Quaternions, 3 positions
        b_x, b_y, b_z = tuple(soccer_current_pose[4:])
        ball_XZ  = np.array([soccer_current_pose[4], soccer_current_pose[6]])



        intercepted = np.sqrt((e_x - b_x)**2 + (e_y - b_y)**2 + (e_z- 0.15 - b_z)**2)  < .30
        solved = False


        try: 
            self.simulator.AdvanceTo(self.sim_time_step + self.simulator.get_context().get_time())
        except:
            self.simulator.AdvanceTo(self.sim_time_step*2 + self.simulator.get_context().get_time())

     
        
        return end_effector_XZ, ball_XZ, intercepted, rpy

### Training,  Loading Model and Testing:

In [5]:
import gym
import numpy as np

from stable_baselines3 import SAC

env = SoccerEnv()

model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=500, log_interval=4)
model.save("SAC_Soc")

del model # remove to demonstrate saving and loading

model = SAC.load("SAC_Soc")

obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    
    if done:
        env.render()
        obs = env.reset()

/usr/local/lib/python3.8/dist-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Total Ep. Reward:  0
Total Ep. Reward:  -500.71255757915856
Total Ep. Reward:  -310.1558736630501
Total Ep. Reward:  -566.0136865131228
Total Ep. Reward:  -536.4092458139659
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46       |
|    ep_rew_mean     | -478     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 102      |
|    time_elapsed    | 1        |
|    total_timesteps | 184      |
| train/             |          |
|    actor_loss      | 2.29     |
|    critic_loss     | 8.19     |
|    ent_coef        | 0.976    |
|    ent_coef_loss   | -0.289   |
|    learning_rate   | 0.0003   |
|    n_updates       | 83  

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c79dd6be-c791-4b96-b0a1-e37f965290d0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>